<div style="text-align: center; font-family: 'charter bt pro roman'; color: rgb(0, 65, 75);">
    <h1>
    GDP Vintages, Releases and Revisions datasets
    </h1>
</div>

<div style="text-align: center; font-family: 'charter bt pro roman'; color: rgb(0, 65, 75);">
<h3>
Documentation
<br>
____________________
<br>
</h3>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    This 
    <span style="color: rgb(0, 65, 75);">jupyter notebook</span>
    provides a step-by-step guide to <b>data building</b> regarding the project <b>'Revisiones y sesgos en las estimaciones preliminares del PBI en el Perú'</b>. This guide covers the creation of GDP mid-term revision dataset for each sector. A key step is the construction at par of what we will call “The ‘t+h’ structure”. This dataset is similar to that of the GDP growth vintages by sector, but instead of growth rate values, it contains values of type “t+h”, where h indicates how many months have passed since the preliminary growth rate was first published; that is, this jupyter notebook also covers the creation of vintages datasets of growth rates associated with a horizon (<b>h</b>).
</div>

<div style="text-align: center; font-family: 'PT Serif Pro Book'; color: rgb(0, 65, 75); font-size: 16px;">
    Jason Cruz
    <br>
    <a href="mailto:jj.cruza@up.edu.pe" style="color: rgb(0, 153, 123); font-size: 16px;">
        jj.cruza@up.edu.pe
    </a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;line-height: 1.5;">
<span style="font-size: 34px;">&#128452;</span> The 't+h' structure should be available for all sectors and frequencies.
    <br>
    <span style="font-size: 24px;">&#8987;</span> Available since <b>1994-2024</b> (Table 1) and since <b>1997-2024</b> (Table 2). 
    <br>
</div>

<div style="font-family: Amaya; text-align: left; color: rgb(0, 65, 75); font-size:16px">The following <b>outline is functional</b>. By utilising the provided buttons, users are able to enhance their experience by browsing this script.<div/>

<div id="outilne">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #292929; padding: 10px; line-height: 1.5; font-family: 'PT Serif Pro Book';">
    <h2 style="text-align: left; color: #E0E0E0;">
        Outline
    </h2>
    <br>
    <a href="#libraries" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        Libraries</a>
    <br>
    <a href="#setup" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        Initial set-up</a>
    <br>
    <a href="#1" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        1. Economic sector selector</a>
    <br>
    <a href="#2" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        2. Create horizon datasets</a>
    <br>
    <a href="#2.1." style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        2.1. Loading growth rate datasets from postgresql.</a>
    <br>
    <a href="#2.2." style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        2.2. Creating horizon dataset step by step.</a> 
    <br>
    <a href="#3" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        3. Create base year datasets</a>
    <br>
    <a href="#3.1." style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        3.1. Loading growth rate datasets from postgresql.</a>
    <br>
    <a href="#3.2." style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        3.2. Creating base year dataset.</a> 
    <br>
    <a href="#4" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        4. Remove observations affected by base year</a>
    <br>
    <a href="#5" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        5. Create growth rates by horizon dataset</a>
    <br>
    <a href="#6" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        6. Create vintages and releases datasets</a>
    <br>
    <a href="#7" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        7. Loading to SQL</a>
    <br>
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Any questions or issues regarding the coding, please email Jason Cruz <a href="mailto:jj.cruza@alum.up.edu.pe" style="color: rgb(0, 153, 123); text-decoration: none;"><span style="font-size: 24px;">&#x2709;</span>
    </a>.
    <div/>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    If you don't have the libraries below, please use the following code (as example) to install the required libraries.
    <div/>

In [ ]:
#!pip install os # Comment this code with "#" if you have already installed this library.

<div id="libraries">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark;">
    <h2>
    Libraries
    </h2>
    <div/>

In [1]:
# POSTGRESSQL
import os
from sqlalchemy import create_engine

# HORIZON DATASETS
import pandas as pd
import numpy as np
import re


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="setup">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark;">
    <h2>
    Initial set-up
    </h2>
    <div/>

<p style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> The following function will establish a connection to the <code>gdp_revisions_datasets</code> database in <code>PostgreSQL</code>. The <b>input data</b> used in this jupyter notebook will be loaded from this <code>PostgreSQL</code> database, and similarly, all <b>output data</b> generated by this jupyter notebook will be stored in that database. Ensure that you set the necessary parameters to access the server once you have obtained the required permissions.<p/>
    
<p style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
To request permissions, please email Jason Cruz <a href="mailto:jj.cruza@alum.up.edu.pe" style="color: rgb(0, 153, 123); text-decoration: none;"> <span style="font-size: 24px;">&#x2709;</span>
    </a>.
<p/>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    <span style="font-size: 24px; color: #FFA823; font-weight: bold;">&#9888;</span>
    Enter your user credentials to acces to SQL.
    <div/>

In [2]:
def create_sqlalchemy_engine():
    """
    Function to create an SQLAlchemy engine using environment variables.
    
    Returns:
        engine: SQLAlchemy engine object.
    """
    # Get environment variables
    user = os.environ.get('CIUP_SQL_USER')  # Get the SQL user from environment variables
    password = os.environ.get('CIUP_SQL_PASS')  # Get the SQL password from environment variables
    host = os.environ.get('CIUP_SQL_HOST')  # Get the SQL host from environment variables
    port = 5432  # Set the SQL port to 5432
    database = 'gdp_revisions_datasets'  # Set the database name 'gdp_revisions_datasets' from SQL

    # Check if all environment variables are defined
    if not all([host, user, password]):
        raise ValueError("Some environment variables are missing (CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS)")

    # Create connection string
    connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

    # Create SQLAlchemy engine
    engine = create_engine(connection_string)
    
    return engine

<div style="text-align: left;">
    <span style="font-size: 24px; color: rgb(255, 32, 78); font-weight: bold;">&#9888;</span>
    <span style="font-family: PT Serif Pro Book; color: black; font-size: 16px;">
        Import all other functions required by this jupyter notebook.
    </span>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Please, check the script <code>gdp_revisions_datasets_functions.py</code> which contains all the functions required by this jupyter notebook. The functions there are ordered according to the <a href="#outilne" style="color: #3d30a2;">sections</a> of this jupyter notebok.<div/>

In [3]:
from gdp_revisions_datasets_functions import *

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="1">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book; color: dark;">1.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Economic sector and data frequency selector</span></h1>

<div id="steps-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#step-1-1" style="text-decoration: none; color: #006769"> <span style="font-size: 24px; color: rgb(0, 65, 75);">&#10122;</span> Select economic sector</a>
    <br>
    <a href="#step-1-2" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> Select frequency</a>
</div>

<div id="step-1-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10122;</span> <span>Select economic sector</span>
  </div>

In [586]:
# Call the function to show the popup window
sector = show_option_window()
print("Selected economic sector:", sector)

Selected economic sector: services


<div id="step-1-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> <span>Select frequency</span>
  </div>

In [587]:
# Call the function to show the popup window
frequency = show_frequency_window()
print("Selected frequency:", frequency)

Selected frequency: annual


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-1" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-1" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="2">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book; color: dark;">2.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Create horizon datasets</span></h1>

<div id="2.1.">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">2.1. </span> <span style = "color: dark; font-family: PT Serif Pro Book;">Loading growth rate datasets from <code>PostgresSQL</code></span></h2>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Connect to SQL.
    <div/>

In [588]:
# Connect to SQL
engine = create_sqlalchemy_engine()

# SQL Query
query = f"SELECT * FROM {sector}_{frequency}_growth_rates;" # Please change your query to PosgtresSQL as you see fit

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Comment the code below if you want the default option (display rows and columns of the dataframe in a limited way)
    <div/>

In [589]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Check the loaded dataframe.
    <div/>

In [590]:
# Read growth rates dataset as DataFrame
globals()[f'{sector}_{frequency}_growth_rates'] = pd.read_sql(query, engine)
growth_rates = globals()[f'{sector}_{frequency}_growth_rates']
growth_rates_df = growth_rates.copy()
growth_rates_df.head(10)

,year,id_ns,date,year_1993,year_1994,year_1995,year_1996,year_1997,year_1998,year_1999,year_2000,year_2001,year_2002,year_2003,year_2004,year_2005,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020,year_2021,year_2022,year_2023
0,1997,24,1997-06-27,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1997,25,1997-07-04,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1997,26,1997-07-11,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1997,27,1997-07-18,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1997,28,1997-07-25,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1997,29,1997-08-08,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1997,30,1997-08-15,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1997,31,1997-08-22,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1997,32,1997-08-29,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1997,33,1997-09-05,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [591]:
# Find duplicates of id_ns within the same year
duplicated_rows = growth_rates_df[growth_rates_df.duplicated(subset=['year', 'id_ns'], keep=False)]
duplicated_rows

,year,id_ns,date,year_1993,year_1994,year_1995,year_1996,year_1997,year_1998,year_1999,year_2000,year_2001,year_2002,year_2003,year_2004,year_2005,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020,year_2021,year_2022,year_2023


In [592]:
growth_rates_df.iloc[:10,:5]

,year,id_ns,date,year_1993,year_1994
0,1997,24,1997-06-27,None,None
1,1997,25,1997-07-04,None,None
2,1997,26,1997-07-11,None,None
3,1997,27,1997-07-18,None,None
4,1997,28,1997-07-25,None,None
5,1997,29,1997-08-08,None,None
6,1997,30,1997-08-15,None,None
7,1997,31,1997-08-22,None,None
8,1997,32,1997-08-29,None,None
9,1997,33,1997-09-05,None,None


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="2.2.">
   <!-- Contenido de la celda de destino -->
</div>

<div id="2.3.">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">2.2.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    Creating horizon dataset step by step
    </span>
    </h2>

<div id="steps-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#step-2-1" style="text-decoration: none; color: #006769"> <span style="font-size: 24px; color: rgb(0, 65, 75);">&#10122;</span> Replace decimal values by “t+h” values only in the rows representing a new rung</a>
    <br>
    <a href="#step-2-2" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> Concatenate first 3 columns: year, date, id_ns </a>
    <br>
    <a href="#step-2-3" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> Convert columns to string type</a>
    <br>
    <a href="#step-2-4" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> Spreads the "t+h" values over the remaining decimal values </a>
    <br>
    <a href="#step-2-5" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> Exporting to excel file </a>
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
   <span>Set key variables to fill the growth_rates with 't+h' values</span>
  </div>

In [593]:
# Call the function to show the h_initial window
h_initial = show_h_initial_window()
print("Selected h_initial:", h_initial)

Selected h_initial: 12


In [594]:
# Call the function to show the start_row window
start_row = show_start_row_window()
print("Selected start_row:", start_row)

Selected start_row: 0


In [595]:
# Define the mapping of frequencies to h_counter values
frequency_mapping = {
    'monthly': 1,
    'quarterly': 3,
    'annual': 12
}

# Get the appropriate h_counter value based on the selected frequency
h_counter = frequency_mapping.get(frequency)
print("Selected h_counter:", h_counter)

Selected h_counter: 12


<div id="step-2-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10122;</span> <span>Replace decimal values by “t+h” values only in the rows representing a new rung</span>
  </div>

In [596]:
horizon = replace_horizon(growth_rates_df.iloc[:, 3:], start_row, h_initial, h_counter)
horizon_df = horizon.copy()
horizon_df.head(10)

,year_1993,year_1994,year_1995,year_1996,year_1997,year_1998,year_1999,year_2000,year_2001,year_2002,year_2003,year_2004,year_2005,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020,year_2021,year_2022,year_2023
0,None,None,t+24,t+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<div id="step-2-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> <span>Concatenate first 3 columns: year, date, id_ns</span>
  </div>

In [597]:
# Get the first three columns of the original DataFrame
first_3_columns = growth_rates_df.iloc[:, :3]

# Concatenate the first three columns with h_{sector}_{frequency}_growth_rates
horizon_df = pd.concat([first_3_columns, horizon_df], axis=1)
horizon_df.head(20)

,year,id_ns,date,year_1993,year_1994,year_1995,year_1996,year_1997,year_1998,year_1999,year_2000,year_2001,year_2002,year_2003,year_2004,year_2005,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020,year_2021,year_2022,year_2023
0,1997,24,1997-06-27,None,None,t+24,t+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1997,25,1997-07-04,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1997,26,1997-07-11,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1997,27,1997-07-18,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1997,28,1997-07-25,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1997,29,1997-08-08,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1997,30,1997-08-15,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1997,31,1997-08-22,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1997,32,1997-08-29,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1997,33,1997-09-05,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<div id="step-2-3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> <span>Convert columns to string type</span>
  </div>

In [598]:
horizon_df = columns_str(horizon_df)
horizon_df.head(20)

,year,id_ns,date,year_1993,year_1994,year_1995,year_1996,year_1997,year_1998,year_1999,year_2000,year_2001,year_2002,year_2003,year_2004,year_2005,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020,year_2021,year_2022,year_2023
0,1997,24,1997-06-27,,,t+24,t+12,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1997,25,1997-07-04,,,7.0,2.9,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,1997,26,1997-07-11,,,7.0,2.9,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,1997,27,1997-07-18,,,7.0,2.9,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,1997,28,1997-07-25,,,7.0,2.9,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,1997,29,1997-08-08,,,7.0,2.9,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,1997,30,1997-08-15,,,7.0,2.9,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,1997,31,1997-08-22,,,7.0,2.9,,,,,,,,,,,,,,,,,,,,,,,,,,,
8,1997,32,1997-08-29,,,7.0,2.9,,,,,,,,,,,,,,,,,,,,,,,,,,,
9,1997,33,1997-09-05,,,7.0,2.9,,,,,,,,,,,,,,,,,,,,,,,,,,,


<div id="step-2-4">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> Spreads the "t+h" values over the remaining decimal values
  </div>

In [599]:
horizon_df = replace_horizon_1(horizon_df)
horizon_df.head(20)

,year,id_ns,date,year_1993,year_1994,year_1995,year_1996,year_1997,year_1998,year_1999,year_2000,year_2001,year_2002,year_2003,year_2004,year_2005,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020,year_2021,year_2022,year_2023
0,1997,24,1997-06-27,,,t+24,t+12,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1997,25,1997-07-04,,,t+25,t+13,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,1997,26,1997-07-11,,,t+25,t+13,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,1997,27,1997-07-18,,,t+25,t+13,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,1997,28,1997-07-25,,,t+25,t+13,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,1997,29,1997-08-08,,,t+26,t+14,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,1997,30,1997-08-15,,,t+26,t+14,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,1997,31,1997-08-22,,,t+26,t+14,,,,,,,,,,,,,,,,,,,,,,,,,,,
8,1997,32,1997-08-29,,,t+26,t+14,,,,,,,,,,,,,,,,,,,,,,,,,,,
9,1997,33,1997-09-05,,,t+27,t+15,,,,,,,,,,,,,,,,,,,,,,,,,,,


<div id="step-2-5">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> Exporting to excel file
  </div>

In [600]:
#def export_to_excel(df, filename):
#    # Exportar el DataFrame como un archivo Excel
#    df.to_excel(filename, index=False)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-2" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-2" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="3">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Create base year datasets</span></h1>

<div id="3.1.">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.1. </span> <span style = "color: dark; font-family: PT Serif Pro Book;">Loading growth rate datasets from <code>PostgresSQL</code></span></h2>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Connect to SQL.
    <div/>

In [601]:
# SQL Query
query = f"SELECT * FROM ns_base_year;" # Please change your query to PosgtresSQL as you see fit

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Check the loaded dataframe.
    <div/>

In [602]:
# Read growth rates dataset as DataFrame
ns_base_year = pd.read_sql(query, engine)
ns_base_year_df = ns_base_year.copy()
ns_base_year_df.head(10)

,year,id_ns,date,base_year
0,1994,01,1994-01-10,1990
1,1994,02,1994-01-17,1990
2,1994,03,1994-01-24,1990
3,1994,04,1994-02-01,1990
4,1994,05,1994-02-07,1990
5,1994,06,1994-02-14,1990
6,1994,07,1994-02-21,1990
7,1994,08,1994-02-28,1990
8,1994,09,1994-03-04,1990
9,1994,10,1994-03-14,1990


<div id="3.2.">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.2.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    Creating base year dataset
    </span>
    </h2>

In [603]:
base_year_df = replace_floats_with_base_year(ns_base_year_df, growth_rates_df)

In [604]:
base_year_df.head(10)

,year,id_ns,date,year_1993,year_1994,year_1995,year_1996,year_1997,year_1998,year_1999,year_2000,year_2001,year_2002,year_2003,year_2004,year_2005,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020,year_2021,year_2022,year_2023
0,1997,24,1997-06-27,None,None,1990,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1997,25,1997-07-04,None,None,1990,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1997,26,1997-07-11,None,None,1990,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1997,27,1997-07-18,None,None,1990,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1997,28,1997-07-25,None,None,1990,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1997,29,1997-08-08,None,None,1990,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1997,30,1997-08-15,None,None,1990,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1997,31,1997-08-22,None,None,1990,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1997,32,1997-08-29,None,None,1990,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1997,33,1997-09-05,None,None,1990,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Consider the code below as provisional. Actually, no data should be exported to the current directory folder, all data should be uploaded to SQL.
    <div/>

In [605]:
# Export to excel file
#with pd.ExcelWriter('base_year_df.xlsx') as writer:
#    base_year_df.to_excel(writer, sheet_name='base_year_df', index=False) # this is an optional, to view data generated

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="4">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">4.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Remove observations affected by base year</span></h1>

<div id="steps-4">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#st-4-1" style="text-decoration: none; color: #006769"> <span style="font-size: 24px; color: rgb(0, 65, 75);">&#10122;</span> Generating a dictionary to match observations affected by base year</a>
    <br>
    <a href="#st-4-2" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> Remove observations affected by base year</a>
    <br>
</div>

<div id="st-4-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10122;</span> <span>Generating a dictionary to match observations affected by base year</span>
  </div>

In [606]:
base_year_dictionary = create_dic_base_year(base_year_df)

In [607]:
base_year_dictionary

{'year_1998': {151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  159,
  160,
  161,
  162,
  163,
  164,
  165,
  166,
  167,
  168,
  169,
  170,
  171,
  172,
  173,
  174,
  175,
  176,
  177,
  178,
  179,
  180,
  181,
  182,
  183,
  184,
  185,
  186,
  187,
  188,
  189,
  190,
  191},
 'year_1999': {151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  159,
  160,
  161,
  162,
  163,
  164,
  165,
  166,
  167,
  168,
  169,
  170,
  171,
  172,
  173,
  174,
  175,
  176,
  177,
  178,
  179,
  180,
  181,
  182,
  183,
  184,
  185,
  186,
  187,
  188,
  189,
  190,
  191,
  192,
  193,
  194,
  195,
  196,
  197,
  198,
  199,
  200,
  201,
  202,
  203,
  204,
  205,
  206,
  207,
  208,
  209,
  210,
  211,
  212,
  213,
  214,
  215,
  216,
  217,
  218,
  219,
  220,
  221,
  222,
  223,
  224,
  225,
  226,
  227,
  228,
  229,
  230,
  231,
  232,
  233,
  234,
  235,
  236,
  237,
  238,
  239},
 'year_2011': {823, 824, 825, 826, 827, 828, 829, 830},
 'year

<div id="st-4-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> <span>Remove observations affected by base year</span>
  </div>

In [608]:
input_growth_rates_df = remove_base_year_affected_obs(base_year_dictionary, growth_rates_df)

In [609]:
input_growth_rates_df.head(30)

,year,id_ns,date,year_1993,year_1994,year_1995,year_1996,year_1997,year_1998,year_1999,year_2000,year_2001,year_2002,year_2003,year_2004,year_2005,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020,year_2021,year_2022,year_2023
0,1997,24,1997-06-27,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1997,25,1997-07-04,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1997,26,1997-07-11,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1997,27,1997-07-18,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1997,28,1997-07-25,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1997,29,1997-08-08,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1997,30,1997-08-15,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1997,31,1997-08-22,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1997,32,1997-08-29,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1997,33,1997-09-05,None,None,7.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Consider the code below as provisional. Actually, no data should be exported to the current directory folder, all data should be uploaded to SQL.
    <div/>

In [610]:
# Export to excel file
#with pd.ExcelWriter('input_growth_rates_df.xlsx') as writer:
#    input_growth_rates_df.to_excel(writer, sheet_name='input_growth_rates_df', index=False) # this is an optional, to view data generated

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="5">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">5.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Create growth rates by horizon dataset</span></h1>

<div id="steps-5">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#st-3-1" style="text-decoration: none; color: #006769"> <span style="font-size: 24px; color: rgb(0, 65, 75);">&#10122;</span> Generating a dictionary with the row indices and their t+h values</a>
    <br>
    <a href="#st-3-2" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> Colapse growth rates by horizon ('t+h') </a>
    <br>
</div>

<div id="st-5-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10122;</span> <span>Generating a dictionary with the row indices and their t+h values</span>
  </div>

In [611]:
horizon_dictionary = get_last_index_h(horizon_df)

<div id="st-5-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> <span>Colapse growth rates by horizon ('t+h')  </span>
  </div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Note that <code>'t'</code> is the date of the first announcement, while <code>'t+h'</code> is 'h' months after the first announcement.
    <div/>

In [612]:
filtered_h_df = filter_df_by_indices(input_growth_rates_df, horizon_dictionary)
filtered_h_df

,horizon,year_1993,year_1994,year_1995,year_1996,year_1997,year_1998,year_1999,year_2000,year_2001,year_2002,year_2003,year_2004,year_2005,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020,year_2021,year_2022,year_2023
0,t+1,NaN,NaN,7.0,2.9,6.3,2.1,1.5,2.8,0.4,4.9,4.4,6.0,6.8,8.2,9.1,9.5,3.1,8.1,8.3,7.2,5.8,4.9,4.2,3.9,3.4,4.4,3.8,-10.3,11.8,3.4,-0.4
1,t+2,NaN,NaN,7.0,3.2,6.3,2.1,1.5,2.8,0.4,4.9,4.4,6.0,6.8,8.2,9.1,9.5,3.1,8.0,8.3,7.2,NaN,4.9,4.2,3.9,3.4,4.4,3.8,-10.3,11.8,3.4,-0.4
2,t+3,NaN,NaN,7.0,3.2,6.3,2.1,1.5,2.8,0.4,4.9,4.4,6.0,6.8,8.2,9.1,9.5,3.1,8.0,8.3,7.2,NaN,4.9,4.2,3.9,3.4,4.4,3.8,-10.3,11.8,3.4,-0.3
3,t+4,NaN,NaN,7.0,3.2,6.6,2.1,1.5,2.8,0.4,4.5,4.5,4.7,6.2,8.2,9.1,9.3,3.1,8.0,8.3,7.3,NaN,4.9,4.2,3.9,3.3,4.4,3.8,-10.3,11.9,3.4,-0.4
4,t+5,NaN,NaN,7.0,3.2,6.6,2.1,1.5,2.8,0.4,4.5,4.5,4.7,6.4,8.2,8.9,9.3,3.1,8.0,8.3,7.3,NaN,4.9,4.2,3.9,3.3,4.4,3.8,-10.3,11.9,3.4,-0.4
5,t+6,NaN,NaN,7.0,3.2,6.6,1.6,NaN,1.8,0.4,4.5,4.5,4.7,6.3,8.2,8.9,9.3,3.1,8.0,8.3,7.3,NaN,4.9,4.2,3.9,3.3,4.4,3.8,-10.3,11.9,3.4,NaN
6,t+7,NaN,NaN,7.0,3.2,6.6,1.6,NaN,1.8,0.4,4.5,4.0,4.7,6.3,7.0,9.6,9.3,3.1,8.0,8.3,7.0,NaN,5.0,4.2,4.0,3.3,4.4,3.8,-10.3,11.8,3.4,NaN
7,t+8,NaN,NaN,7.0,3.2,6.6,1.6,NaN,1.8,0.4,4.5,4.0,4.7,6.3,7.0,9.6,9.3,3.1,8.0,8.3,7.0,NaN,5.0,4.2,4.0,3.3,4.4,3.8,-10.3,11.8,3.4,NaN
8,t+9,NaN,NaN,7.0,3.2,6.6,1.6,NaN,1.8,0.4,4.5,4.0,4.7,6.3,7.0,9.6,9.3,3.1,8.0,8.3,7.0,NaN,5.0,4.2,4.0,3.3,4.4,3.8,-10.3,11.8,3.4,NaN
9,t+10,NaN,NaN,7.0,3.2,6.6,1.6,NaN,1.8,0.4,3.8,4.0,4.7,6.3,6.8,8.9,9.3,3.1,8.0,8.3,7.1,NaN,5.0,4.2,4.0,3.3,4.5,3.8,-10.3,11.8,3.3,NaN


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-5" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-5" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="6">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">6.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Create vintages and releases datasets</span></h1>

<div id="steps-6">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#st-4-1" style="text-decoration: none; color: #006769"> <span style="font-size: 24px; color: rgb(0, 65, 75);">&#10122;</span> Create vintages</a>
    <br>
    <a href="#st-4-2" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> Clean-up vintages dataframe </a>
    <br>
    <a href="#st-4-3" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> Create releases </a>
    <br>
</div>

<div id="st-6-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10122;</span> <span>Create vintages</span>
  </div>

In [613]:
vintages = create_vintages(input_growth_rates_df)
vintages.head(30)

aux,vintages_date,ns_1997m06,ns_1997m07,ns_1997m08,ns_1997m09,ns_1997m10,ns_1997m11,ns_1997m12,ns_1998m01,ns_1998m02,ns_1998m03,ns_1998m04,ns_1998m05,ns_1998m06,ns_1998m07,ns_1998m08,ns_1998m09,ns_1998m10,ns_1998m11,ns_1998m12,ns_1999m01,ns_1999m02,ns_1999m03,ns_1999m04,ns_1999m05,ns_1999m06,ns_1999m07,ns_1999m08,ns_1999m09,ns_1999m10,ns_1999m11,ns_1999m12,ns_2000m01,ns_2000m02,ns_2000m03,ns_2000m04,ns_2000m05,ns_2000m06,ns_2000m07,ns_2000m08,ns_2000m09,ns_2000m10,ns_2000m11,ns_2000m12,ns_2001m01,ns_2001m02,ns_2001m03,ns_2001m04,ns_2001m05,ns_2001m06,ns_2001m07,ns_2001m08,ns_2001m09,ns_2001m10,ns_2001m11,ns_2001m12,ns_2002m01,ns_2002m02,ns_2002m03,ns_2002m04,ns_2002m05,ns_2002m06,ns_2002m07,ns_2002m08,ns_2002m09,ns_2002m10,ns_2002m11,ns_2002m12,ns_2003m01,ns_2003m02,ns_2003m03,ns_2003m04,ns_2003m05,ns_2003m06,ns_2003m07,ns_2003m08,ns_2003m09,ns_2003m10,ns_2003m11,ns_2003m12,ns_2004m01,ns_2004m02,ns_2004m03,ns_2004m04,ns_2004m05,ns_2004m06,ns_2004m07,ns_2004m08,ns_2004m09,ns_2004m10,ns_2004m11,ns_2004m12,ns_2005m01,ns_2005m02,ns_2005m03,ns_2005m04,ns_2005m05,ns_2005m06,ns_2005m07,ns_2005m08,ns_2005m09,ns_2005m10,ns_2005m11,ns_2005m12,ns_2006m01,ns_2006m02,ns_2006m03,ns_2006m04,ns_2006m05,ns_2006m06,ns_2006m07,ns_2006m08,ns_2006m09,ns_2006m10,ns_2006m11,ns_2006m12,ns_2007m01,ns_2007m02,ns_2007m03,ns_2007m04,ns_2007m05,ns_2007m06,ns_2007m07,ns_2007m08,ns_2007m09,ns_2007m10,ns_2007m11,ns_2007m12,ns_2008m01,ns_2008m02,ns_2008m03,ns_2008m04,ns_2008m05,ns_2008m06,ns_2008m07,ns_2008m08,ns_2008m09,ns_2008m10,ns_2008m11,ns_2008m12,ns_2009m01,ns_2009m02,ns_2009m03,ns_2009m04,ns_2009m05,ns_2009m06,ns_2009m07,ns_2009m08,ns_2009m09,ns_2009m10,ns_2009m11,ns_2009m12,ns_2010m01,ns_2010m02,ns_2010m03,ns_2010m04,ns_2010m05,ns_2010m06,ns_2010m07,ns_2010m08,ns_2010m09,ns_2010m10,ns_2010m11,ns_2010m12,ns_2011m01,ns_2011m02,ns_2011m03,ns_2011m04,ns_2011m05,ns_2011m06,ns_2011m07,ns_2011m08,ns_2011m09,ns_2011m10,ns_2011m11,ns_2011m12,ns_2012m01,ns_2012m02,ns_2012m03,ns_2012m04,ns_2012m05,ns_2012m06,ns_2012m07,ns_2012m08,ns_2012m09,ns_2012m10,ns_2012m11,ns_2012m12,ns_2013m01,ns_2013m02,ns_2013m03,ns_2013m04,ns_2013m05,ns_2013m06,ns_2013m07,ns_2013m08,ns_2013m09,ns_2013m10,ns_2013m11,ns_2013m12,ns_2014m01,ns_2014m02,ns_2014m03,ns_2014m04,ns_2014m05,ns_2014m06,ns_2014m07,ns_2014m08,ns_2014m09,ns_2014m10,ns_2014m11,ns_2014m12,ns_2015m01,ns_2015m02,ns_2015m03,ns_2015m04,ns_2015m05,ns_2015m06,ns_2015m07,ns_2015m08,ns_2015m09,ns_2015m10,ns_2015m11,ns_2015m12,ns_2016m01,ns_2016m02,ns_2016m03,ns_2016m04,ns_2016m05,ns_2016m06,ns_2016m07,ns_2016m08,ns_2016m09,ns_2016m10,ns_2016m11,ns_2016m12,ns_2017m01,ns_2017m02,ns_2017m03,ns_2017m04,ns_2017m05,ns_2017m06,ns_2017m07,ns_2017m08,ns_2017m09,ns_2017m10,ns_2017m11,ns_2017m12,ns_2018m01,ns_2018m02,ns_2018m03,ns_2018m04,ns_2018m05,ns_2018m06,ns_2018m07,ns_2018m08,ns_2018m09,ns_2018m10,ns_2018m11,ns_2018m12,ns_2019m01,ns_2019m02,ns_2019m03,ns_2019m04,ns_2019m05,ns_2019m06,ns_2019m07,ns_2019m08,ns_2019m09,ns_2019m10,ns_2019m11,ns_2019m12,ns_2020m01,ns_2020m02,ns_2020m03,ns_2020m04,ns_2020m05,ns_2020m06,ns_2020m07,ns_2020m08,ns_2020m09,ns_2020m10,ns_2020m11,ns_2020m12,ns_2021m01,ns_2021m02,ns_2021m03,ns_2021m04,ns_2021m05,ns_2021m06,ns_2021m07,ns_2021m08,ns_2021m09,ns_2021m10,ns_2021m11,ns_2021m12,ns_2022m01,ns_2022m02,ns_2022m03,ns_2022m04,ns_2022m05,ns_2022m06,ns_2022m07,ns_2022m08,ns_2022m09,ns_2022m10,ns_2022m11,ns_2022m12,ns_2023m01,ns_2023m02,ns_2023m03,ns_2023m04,ns_2023m05,ns_2023m06,ns_2023m07,ns_2023m08,ns_2023m09,ns_2023m10,ns_2023m11,ns_2023m12,ns_2024m01,ns_2024m02,ns_2024m03,ns_2024m04,ns_2024m05,ns_2024m06
0,year_1993,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,

<div id="st-6-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> <span>Clean-up vintages dataframe</span>
  </div>

In [614]:
# Call the appropriate function based on frequency and sector
if frequency == 'monthly':
    vintages_df = process_monthly(vintages)
elif frequency == 'quarterly':
    vintages_df = process_quarterly(vintages)
elif frequency == 'annual':
    vintages_df = process_annual(vintages)

In [615]:
vintages_df.head(30)

aux,vintages_date,ns_1997m06,ns_1997m07,ns_1997m08,ns_1997m09,ns_1997m10,ns_1997m11,ns_1997m12,ns_1998m01,ns_1998m02,ns_1998m03,ns_1998m04,ns_1998m05,ns_1998m06,ns_1998m07,ns_1998m08,ns_1998m09,ns_1998m10,ns_1998m11,ns_1998m12,ns_1999m01,ns_1999m02,ns_1999m03,ns_1999m04,ns_1999m05,ns_1999m06,ns_1999m07,ns_1999m08,ns_1999m09,ns_1999m10,ns_1999m11,ns_1999m12,ns_2000m01,ns_2000m02,ns_2000m03,ns_2000m04,ns_2000m05,ns_2000m06,ns_2000m07,ns_2000m08,ns_2000m09,ns_2000m10,ns_2000m11,ns_2000m12,ns_2001m01,ns_2001m02,ns_2001m03,ns_2001m04,ns_2001m05,ns_2001m06,ns_2001m07,ns_2001m08,ns_2001m09,ns_2001m10,ns_2001m11,ns_2001m12,ns_2002m01,ns_2002m02,ns_2002m03,ns_2002m04,ns_2002m05,ns_2002m06,ns_2002m07,ns_2002m08,ns_2002m09,ns_2002m10,ns_2002m11,ns_2002m12,ns_2003m01,ns_2003m02,ns_2003m03,ns_2003m04,ns_2003m05,ns_2003m06,ns_2003m07,ns_2003m08,ns_2003m09,ns_2003m10,ns_2003m11,ns_2003m12,ns_2004m01,ns_2004m02,ns_2004m03,ns_2004m04,ns_2004m05,ns_2004m06,ns_2004m07,ns_2004m08,ns_2004m09,ns_2004m10,ns_2004m11,ns_2004m12,ns_2005m01,ns_2005m02,ns_2005m03,ns_2005m04,ns_2005m05,ns_2005m06,ns_2005m07,ns_2005m08,ns_2005m09,ns_2005m10,ns_2005m11,ns_2005m12,ns_2006m01,ns_2006m02,ns_2006m03,ns_2006m04,ns_2006m05,ns_2006m06,ns_2006m07,ns_2006m08,ns_2006m09,ns_2006m10,ns_2006m11,ns_2006m12,ns_2007m01,ns_2007m02,ns_2007m03,ns_2007m04,ns_2007m05,ns_2007m06,ns_2007m07,ns_2007m08,ns_2007m09,ns_2007m10,ns_2007m11,ns_2007m12,ns_2008m01,ns_2008m02,ns_2008m03,ns_2008m04,ns_2008m05,ns_2008m06,ns_2008m07,ns_2008m08,ns_2008m09,ns_2008m10,ns_2008m11,ns_2008m12,ns_2009m01,ns_2009m02,ns_2009m03,ns_2009m04,ns_2009m05,ns_2009m06,ns_2009m07,ns_2009m08,ns_2009m09,ns_2009m10,ns_2009m11,ns_2009m12,ns_2010m01,ns_2010m02,ns_2010m03,ns_2010m04,ns_2010m05,ns_2010m06,ns_2010m07,ns_2010m08,ns_2010m09,ns_2010m10,ns_2010m11,ns_2010m12,ns_2011m01,ns_2011m02,ns_2011m03,ns_2011m04,ns_2011m05,ns_2011m06,ns_2011m07,ns_2011m08,ns_2011m09,ns_2011m10,ns_2011m11,ns_2011m12,ns_2012m01,ns_2012m02,ns_2012m03,ns_2012m04,ns_2012m05,ns_2012m06,ns_2012m07,ns_2012m08,ns_2012m09,ns_2012m10,ns_2012m11,ns_2012m12,ns_2013m01,ns_2013m02,ns_2013m03,ns_2013m04,ns_2013m05,ns_2013m06,ns_2013m07,ns_2013m08,ns_2013m09,ns_2013m10,ns_2013m11,ns_2013m12,ns_2014m01,ns_2014m02,ns_2014m03,ns_2014m04,ns_2014m05,ns_2014m06,ns_2014m07,ns_2014m08,ns_2014m09,ns_2014m10,ns_2014m11,ns_2014m12,ns_2015m01,ns_2015m02,ns_2015m03,ns_2015m04,ns_2015m05,ns_2015m06,ns_2015m07,ns_2015m08,ns_2015m09,ns_2015m10,ns_2015m11,ns_2015m12,ns_2016m01,ns_2016m02,ns_2016m03,ns_2016m04,ns_2016m05,ns_2016m06,ns_2016m07,ns_2016m08,ns_2016m09,ns_2016m10,ns_2016m11,ns_2016m12,ns_2017m01,ns_2017m02,ns_2017m03,ns_2017m04,ns_2017m05,ns_2017m06,ns_2017m07,ns_2017m08,ns_2017m09,ns_2017m10,ns_2017m11,ns_2017m12,ns_2018m01,ns_2018m02,ns_2018m03,ns_2018m04,ns_2018m05,ns_2018m06,ns_2018m07,ns_2018m08,ns_2018m09,ns_2018m10,ns_2018m11,ns_2018m12,ns_2019m01,ns_2019m02,ns_2019m03,ns_2019m04,ns_2019m05,ns_2019m06,ns_2019m07,ns_2019m08,ns_2019m09,ns_2019m10,ns_2019m11,ns_2019m12,ns_2020m01,ns_2020m02,ns_2020m03,ns_2020m04,ns_2020m05,ns_2020m06,ns_2020m07,ns_2020m08,ns_2020m09,ns_2020m10,ns_2020m11,ns_2020m12,ns_2021m01,ns_2021m02,ns_2021m03,ns_2021m04,ns_2021m05,ns_2021m06,ns_2021m07,ns_2021m08,ns_2021m09,ns_2021m10,ns_2021m11,ns_2021m12,ns_2022m01,ns_2022m02,ns_2022m03,ns_2022m04,ns_2022m05,ns_2022m06,ns_2022m07,ns_2022m08,ns_2022m09,ns_2022m10,ns_2022m11,ns_2022m12,ns_2023m01,ns_2023m02,ns_2023m03,ns_2023m04,ns_2023m05,ns_2023m06,ns_2023m07,ns_2023m08,ns_2023m09,ns_2023m10,ns_2023m11,ns_2023m12,ns_2024m01,ns_2024m02,ns_2024m03,ns_2024m04,ns_2024m05,ns_2024m06
0,1993-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

<div id="st-6-3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> <span>Create releases</span>
  </div>

In [616]:
releases_df = create_releases(vintages_df, sector)
releases_df.head(30)

aux,vintages_date,services_release_1,services_release_2,services_release_3,services_release_4,services_release_5,services_release_6,services_release_7,services_release_8,services_release_9,services_release_10,services_release_11,services_release_12,services_release_13,services_release_14,services_release_15,services_release_16,services_release_17,services_release_18,services_release_19,services_release_20,services_release_21,services_release_22,services_release_23,services_release_24,services_release_25,services_release_26,services_release_27,services_release_28,services_release_29,services_release_30,services_release_31,services_release_32,services_release_33,services_release_34,services_release_35,services_release_36,services_release_37,services_release_38,services_release_39,services_most_recent
0,1993-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1994-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1995-01-01,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
3,1996-01-01,2.9,2.9,2.9,2.9,2.9,2.9,3.2,3.2,3.2,3.2,3.2,3.2,3.2,3.2,3.2,3.2,3.2,3.2,3.2,3.2,3.2,3.2,3.2,3.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.2
4,1997-01-01,6.3,6.3,6.3,6.6,6.6,6.6,6.6,6.6,6.6,6.6,6.6,6.3,6.3,6.3,6.3,6.3,5.3,5.3,5.3,5.3,5.3,5.3,5.3,5.3,5.3,5.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.3
5,1998-01-01,2.1,2.1,2.1,2.1,2.1,1.6,1.6,1.6,1.6,1.6,1.6,1.6,1.5,1.5,1.5,1.5,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5
6,1999-01-01,1.5,1.5,1.5,1.5,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5
7,2000-01-01,2.8,2.8,2.8,2.8,2.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.8
8,2001-01-01,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.3,0.3,0.3,0.5,0.5,0.5,0.5,0.5,0.5,-0.3,-0.3,-0.3,-0.3,-0.3,-0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.3
9,2002-01-01,4.9,4.9,4.9,4.5,4.5,4.5,4.5,4.5,4.5,3.8,3.8,3.8,3.8,3.8,3.8,3.8,3.8,3.8,3.8,3.8,3.8,3.8,3.8,3.8,3.8,3.8,3.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.8


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-6" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

#### Check column type

In [617]:
print(releases_df[f'{sector}_release_1'].dtype)

float64


In [618]:
print(releases_df['vintages_date'].dtype)

datetime64[ns]


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="7">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: charter;">7.</span>
    <span style = "color: dark; font-family: charter;">
    Loading to SQL
    </span>
    </h2>

In [619]:
#horizon_df.to_sql(f'{sector}_{frequency}_growth_rates_horizon', engine, index=False, if_exists='replace')
#base_year_df.to_sql(f'{sector}_{frequency}_growth_rates_base_year', engine, index=False, if_exists='replace')
filtered_h_df.to_sql(f'{sector}_{frequency}_h', engine, index=False, if_exists='replace')
vintages_df.to_sql(f'{sector}_{frequency}_vintages', engine, index=False, if_exists='replace')
releases_df.to_sql(f'{sector}_{frequency}_releases', engine, index=False, if_exists='replace')

31

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-1-1" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-1-1" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select economic sector</b>.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>